In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.preprocessing import StandardScaler
import torch
from torch import optim

In [2]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

In [4]:
print(train.shape)
train = train.drop(train[(train.BMI == 0)].index)
print(train.shape)

(537, 9)
(532, 9)


In [5]:
y = train.Diabetes
x = train.drop('Diabetes', axis=1)

In [6]:
sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [7]:
W = torch.zeros((x.shape[1], 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y.to_numpy())
test_tensor = torch.FloatTensor(test)

In [9]:
loss = torch.nn.BCELoss()

optimizer = optim.SGD([W, b], lr=0.001)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_tensor.matmul(W) + b) # or .mm or @


    ## 중간 생략 

    cost = loss(hypothesis, y_tensor.unsqueeze(1))

    # cost = -(y_tensor * torch.log(hypothesis) + 
            #  (1 - y_tensor) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/10000 Cost: 0.693147
Epoch  100/10000 Cost: 0.679658
Epoch  200/10000 Cost: 0.667246
Epoch  300/10000 Cost: 0.655816
Epoch  400/10000 Cost: 0.645277
Epoch  500/10000 Cost: 0.635547
Epoch  600/10000 Cost: 0.626550
Epoch  700/10000 Cost: 0.618218
Epoch  800/10000 Cost: 0.610492
Epoch  900/10000 Cost: 0.603313
Epoch 1000/10000 Cost: 0.596635
Epoch 1100/10000 Cost: 0.590410
Epoch 1200/10000 Cost: 0.584599
Epoch 1300/10000 Cost: 0.579166
Epoch 1400/10000 Cost: 0.574079
Epoch 1500/10000 Cost: 0.569308
Epoch 1600/10000 Cost: 0.564827
Epoch 1700/10000 Cost: 0.560612
Epoch 1800/10000 Cost: 0.556643
Epoch 1900/10000 Cost: 0.552898
Epoch 2000/10000 Cost: 0.549362
Epoch 2100/10000 Cost: 0.546018
Epoch 2200/10000 Cost: 0.542853
Epoch 2300/10000 Cost: 0.539852
Epoch 2400/10000 Cost: 0.537005
Epoch 2500/10000 Cost: 0.534300
Epoch 2600/10000 Cost: 0.531727
Epoch 2700/10000 Cost: 0.529279
Epoch 2800/10000 Cost: 0.526945
Epoch 2900/10000 Cost: 0.524720
Epoch 3000/10000 Cost: 0.522596
Epoch 31

In [10]:
torch.matmul(test_tensor, W) + b

tensor([[-0.6127],
        [-1.2473],
        [-1.6393],
        [-1.1678],
        [ 0.0551],
        [ 0.0721],
        [-3.3080],
        [-0.0300],
        [ 0.1619],
        [ 1.0093],
        [-0.9088],
        [ 1.6611],
        [-0.2424],
        [-0.5388],
        [-2.0372],
        [-0.4655],
        [-1.4883],
        [-1.9251],
        [ 0.8048],
        [ 0.2024],
        [-0.9713],
        [-1.8372],
        [-0.0211],
        [-1.7617],
        [ 0.2723],
        [ 1.4523],
        [-1.7050],
        [-2.6517],
        [-0.6549],
        [-1.4345],
        [ 1.6883],
        [ 1.5223],
        [ 1.1614],
        [ 0.6259],
        [ 0.5647],
        [ 0.7590],
        [ 2.0652],
        [-0.8917],
        [ 0.0686],
        [ 0.2328],
        [-2.0728],
        [ 0.1333],
        [ 0.2234],
        [-0.4693],
        [-2.5670],
        [ 0.2187],
        [ 0.6229],
        [-1.0466],
        [-0.3993],
        [ 2.6736],
        [-2.4223],
        [ 0.7666],
        [ 1.

In [11]:
submit = pd.read_csv('submit.csv')
submit.Diabetes = [int(x) for x in torch.sigmoid(test_tensor.matmul(W) + b).detach() > 0.6]
submit.to_csv('submission.csv', index=False)